In [1]:
from llm.llm.chatgpt import ChatGPT
from llm.llm.gemini import Gemini
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from llm_test import reasoning_text2SQL, CoT_reasoning
from llm_general import  find_suitable_row_v2

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = Gemini()

In [3]:
question = "For Vietcombank and Techcombank, please provide the quarterly data for the Debt-to-Equity ratio from 2020 to 2023"

In [4]:
find_suitable_row_v2(llm, question, verbose=True)

2024-11-07 16:27:06,689 - INFO - Completion time of gemini-1.5-flash-002: 1.8015248775482178s


prompt_token_count: 197
candidates_token_count: 51
total_token_count: 248

Find suitable column response: 
```json
{
  "industry": ["Banking"],
  "financial_statement_row": ["Total Liabilities", "Total Equity"],
  "financial_ratio_row": ["Debt-to-Equity Ratio"]
}
```



{'industry': ['Banking'],
 'financial_statement_row': ['Total Liabilities', 'Total Equity'],
 'financial_ratio_row': ['Debt-to-Equity Ratio']}

In [5]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [6]:
conn = connect_to_db(db_name, user, password, host, port)


Connecting to database test_db, postgres...


In [7]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_sec_chroma'
persist_directory = 'data/category_sec_chroma'
sec_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_ratio_chroma'
persist_directory = 'data/category_ratio_chroma'
ratio_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

2024-11-07 16:27:07,357 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 16:27:07,507 - INFO - Collection category_bank_chroma is not created.
2024-11-07 16:27:08,119 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 16:27:08,127 - INFO - Collection category_non_bank_chroma is not created.
2024-11-07 16:27:08,748 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 16:27:08,755 - INFO - Collection category_sec_chroma is not created.
2024-11-07 16:27:09,311 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-11-07 16:27:09,320 - INFO - Collection category_ratio_chroma is not created.
2024-11-07 16:27:09,921 - INFO - Anonymized telemetry enabled. See                     h

In [8]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, sec_vector_store, ratio_vector_store, vector_db_company, vector_db_sql)

In [9]:
prompt = "For Vietcombank and Techcombank, please provide the quarterly data for the Debt-to-Equity ratio from 2020 to 2023"

In [10]:
find_suitable_row_v2(llm, prompt, verbose=True)

2024-11-07 16:27:11,352 - INFO - Completion time of gemini-1.5-flash-002: 0.8373923301696777s


prompt_token_count: 197
candidates_token_count: 51
total_token_count: 248

Find suitable column response: 
```json
{
  "industry": ["Banking"],
  "financial_statement_row": ["Total Liabilities", "Total Equity"],
  "financial_ratio_row": ["Debt-to-Equity Ratio"]
}
```



{'industry': ['Banking'],
 'financial_statement_row': ['Total Liabilities', 'Total Equity'],
 'financial_ratio_row': ['Debt-to-Equity Ratio']}

In [11]:
from llm_general import *
import time

In [12]:
get_stock_code_based_on_company_name(llm, prompt, db, verbose=True)

2024-11-07 16:27:11,992 - INFO - Completion time of gemini-1.5-flash-002: 0.6146626472473145s


prompt_token_count: 126
candidates_token_count: 34
total_token_count: 160

Get stock code based on company name response: 
```json
{
  "industry": ["banking"],
  "company_name": ["Vietcombank", "Techcombank"]
}
```



2024-11-07 16:27:12,782 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:13,729 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


(  stock_code                                       company_name  \
 0        CTG  Ngân hàng Thương mại Cổ phần Công thương Việt Nam   
 1        VCB  Ngân hàng Thương mại Cổ phần Ngoại thương Việt...   
 2        TCB    Ngân hàng Thương mại Cổ phần Kỹ thương Việt Nam   
 
                                      en_company_name industry  is_bank  \
 0  Vietnam Joint Stock Commercial Bank For Indust...  Banking     True   
 1  Joint Stock Commercial Bank For Foreign Trade ...  Banking     True   
 2  Vietnam Technological And Commercial Joint Sto...  Banking     True   
 
    is_securities  
 0          False  
 1          False  
 2          False  ,
 ['banking'])

In [13]:
llm.reset_token()
llm.usage()

{'input_token': 0, 'output_token': 0}

In [14]:
start = time.time()
history, error_messages, execution_tables = reasoning_text2SQL(llm, prompt, db, 5, verbose=False)
end = time.time()
print("Time taken: ", end - start)

2024-11-07 16:27:14,491 - INFO - Completion time of gemini-1.5-flash-002: 0.7267031669616699s


prompt_token_count: 188
candidates_token_count: 68
total_token_count: 256



2024-11-07 16:27:15,137 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:15,912 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:16,422 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:16,983 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:17,481 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:18,048 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:18,497 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:19,045 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:20,188 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-11-07 16:27:20,730 - INFO - HTTP

prompt_token_count: 2478
candidates_token_count: 159
total_token_count: 2637

[{'language': 'sql', 'code': "SELECT \n    fr.stock_code,\n    fr.year,\n    fr.quarter,\n    fr.data AS debt_to_equity_ratio\nFROM financial_ratio fr\nJOIN map_category_code_ratio mcr ON fr.ratio_code = mcr.ratio_code\nWHERE mcr.ratio_name = 'Debt To Equity Ratio'\n  AND fr.stock_code IN ('VCB', 'TCB')\n  AND fr.year BETWEEN 2020 AND 2023\n  AND fr.quarter != 0 --Selecting quarterly data, so quarter should not be 0\nORDER BY fr.stock_code, fr.year, fr.quarter;\n"}]
Time taken:  8.390519380569458


In [15]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query.
    
user
_____________
You have the following database schema:
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial reports, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).



Here are the descriptions of tables in the database:



### PostgreSQL tables, with their properties

```sql 

-- Table: company_info

CREATE TABLE company_info(

    stock_code VARCHAR(255) primary key, --The trading symbol.

    is_bank BOOLEAN, --Bool checking wh

In [16]:
last_message = history[-1]['content']
print(last_message)

```sql
SELECT 
    fr.stock_code,
    fr.year,
    fr.quarter,
    fr.data AS debt_to_equity_ratio
FROM financial_ratio fr
JOIN map_category_code_ratio mcr ON fr.ratio_code = mcr.ratio_code
WHERE mcr.ratio_name = 'Debt To Equity Ratio'
  AND fr.stock_code IN ('VCB', 'TCB')
  AND fr.year BETWEEN 2020 AND 2023
  AND fr.quarter != 0 --Selecting quarterly data, so quarter should not be 0
ORDER BY fr.stock_code, fr.year, fr.quarter;
```


### The result of the given SQL:

SQL result for 1: 
| stock_code   |   year |   quarter |   debt_to_equity_ratio |
|:-------------|-------:|----------:|-----------------------:|
| TCB          |   2020 |         1 |                5.066   |
| TCB          |   2020 |         2 |                4.87077 |
| TCB          |   2020 |         3 |                4.68575 |
| TCB          |   2020 |         4 |                4.89163 |
| TCB          |   2021 |         1 |                4.85164 |
| TCB          |   2021 |         2 |                5.01071 |
| TCB

In [17]:
for message in history:
    print("==============")
    print(message['role'])
    print('_____________')
    print(message['content'])

system
_____________

    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query.
    
user
_____________
You have the following database schema:
You're given database about the financial reports of top Vietnamese companies, both bank and cooperates.

All the data in the financial report are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the categories are followed by the International Financial Reporting Standards (IFRS).

There are 3 type of financial reports, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).



Here are the descriptions of tables in the database:



### PostgreSQL tables, with their properties

```sql 

-- Table: company_info

CREATE TABLE company_info(

    stock_code VARCHAR(255) primary key, --The trading symbol.

    is_bank BOOLEAN, --Bool checking wh

In [18]:
llm.usage()

{'input_token': 2666, 'output_token': 227}

In [19]:
execution_tables[-1]

,stock_code,year,quarter,debt_to_equity_ratio
0,TCB,2020,1,5.066004
1,TCB,2020,2,4.870766
2,TCB,2020,3,4.685754
3,TCB,2020,4,4.891633
4,TCB,2021,1,4.851644
5,TCB,2021,2,5.010706
6,TCB,2021,3,5.128175
7,TCB,2021,4,5.112582
8,TCB,2022,1,5.240951
9,TCB,2022,2,4.970413
